In [1]:
from crewai import Agent,Task,Crew,Process,LLM
from key import gooogle_api_key,serper_api_key
from crewai_tools import WebsiteSearchTool,ScrapeWebsiteTool,SerperDevTool

c:\Users\tjwin\anaconda3\envs\crewai_env\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\tjwin\anaconda3\envs\crewai_env\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\tjwin\anaconda3\envs\crewai_env\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are dep

In [2]:
import os
os.environ["GOOGLE_API_KEY"] = gooogle_api_key
os.environ["SERPER_API_KEY"] = serper_api_key

In [12]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults 
import json


@tool
def search_web_tool(query: str):
    """
    Searches the web and returns results.
    """
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)


# search_web_tool = WebsiteSearchTool(
#     config=dict(
#         llm=dict(
#             provider="ollama",
#             config=dict(
#                 model="llama2",
#             ),
#         ),
#         embedder=dict(
#             provider="google",
#             config=dict(
#                 model="models/embedding-001",
#                 task_type="retrieval_document",
#             ),
#         ),
#     )
# )


scraper = ScrapeWebsiteTool()
serper = SerperDevTool()

In [13]:
from crewai import LLM

llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
    #vertex_credentials=vertex_credentials_json
    api_key=gooogle_api_key
)

In [21]:
booking_agent = Agent(
    role = "Booking Specialist",
    goal = "Streamline a flights or trains (which ever will be the best)with in the budget. ",
    backstory = ("""You are skilled in finding the best deals and securing bookings quickly at cheapest."""),
    memory = True,
    allow_deligation = True,
    verbose = True,
    tools = [scraper,serper],
    llm = llm)

In [22]:
travel_planner_ag = Agent(
    role = "Travel Planner",
    goal = "Schedule the complete plan from day to night.",
    backstory = ("An expert in travel planning who knows the best destinations for every type of traveler."),
    tools = [scraper,serper],
    memory = True,
    allow_deligation = True,
    llm = llm ,verbose = True
)

In [ ]:
booking_task = Task(
    description=("""
    This task focuses on streamlining transportation (either flights or trains, whichever is the best) from {start_trip} on {start_date} to {destination} and return date will be next day from the stay. 
    Additionally, it includes finding suitable hotel accommodations in {destination} within the given budget of {Budget} for {people} people with a stay duration of {day} days.

    Key Considerations:
    - Prioritize the cheapest flight/train options while ensuring a balance between cost and convenience.
    - Find budget-friendly and well-rated hotels in {destination}.
    - Ensure all options are properly arranged in a structured format.
    - Provide direct booking links for flights, trains, and hotels.

    """),
    expected_output=f"""
    In markdown format: A structured report that categorizes available options under headings:
    - **Flights/Trains**: Cheapest available transportation options.
    - **Hotels**: Affordable and comfortable stay recommendations.
    
    Ensure that links to booking pages are included for all options.
    """,
    agent=booking_agent,
    tools=[scraper,serper],
)


In [31]:
travel_planner_task = Task(
    description = "streamline the places where user can visit including the main activity at {destination}.",
    agent = travel_planner_ag,
    expected_output = """
    In markdown format: A structured report that categorizes available options under headings:
    - **Flights/Trains**: Cheapest available transportation options with in the table having two rows.
    - **Hotels**: Affordable and comfortable stay recommendations with in the table having two rows.
    - **Schedule**: complete schedule from day to night with eating preferences with time stamps
    
    Ensure that links to booking pages are included for all options.
""",
    output_file="Plan.md"
)

In [32]:
crew_ = Crew(
    agents=[booking_agent,travel_planner_ag],
    tasks=[booking_task,travel_planner_task],
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    verbose=True
)

2025-03-06 18:50:15,714 - 14284 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [35]:
results = crew_.kickoff(inputs={"start_trip":"Chandigarh","destination":"Banglore","Budget" : 30000,"people":2,"day":2,"start_date":05-04-2025})

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (645849508.py, line 1)

In [36]:
from IPython.display import Markdown, display

# Path to your Markdown file


# Open and read the Markdown file
with open("Plan.md", "r", encoding="utf-8") as md_file:
    content = md_file.read()

# Render and display the Markdown content in the notebook
display(Markdown(content))

# Bangalore Trip Plan (Customize Budget and Duration)

This plan provides a framework. Adjust the budget and duration to your preferences.  Remember to book flights and hotels well in advance for better deals.  Prices are estimates and can vary.

## Flights/Trains:

Finding the cheapest flights or trains requires specifying your origin city and travel dates. Use online travel agencies like MakeMyTrip, Goibibo, Expedia, Skyscanner, etc., to compare prices.

| Option        | Booking Link                                                                | Estimated Price (per person, round trip) |
|----------------|-----------------------------------------------------------------------------|-----------------------------------------|
| Flight (Example)| [https://www.makemytrip.com/flights/](https://www.makemytrip.com/flights/) | ₹3000 - ₹6000 (depending on origin and time of booking) |
| Train (Example) | [https://www.irctc.co.in/](https://www.irctc.co.in/)                        | ₹1500 - ₹3000 (depending on origin and class)             |


**Note:**  Train travel can be significantly cheaper than flights, especially for longer distances, but it takes considerably longer.


## Hotels:

Many budget-friendly hotels are available in Bangalore. Use online travel agencies like MakeMyTrip, Goibibo, OYO Rooms, Booking.com etc., to find options that suit your budget and preferences. Read reviews before booking.

| Option        | Booking Link                                                              | Estimated Price (per night, double occupancy) |
|----------------|---------------------------------------------------------------------------|---------------------------------------------|
| Budget Hotel (Example) | [https://www.oyorooms.com/](https://www.oyorooms.com/)                   | ₹1000 - ₹2500                                     |
| Mid-Range Hotel (Example)| [https://www.booking.com/](https://www.booking.com/)                  | ₹3000 - ₹5000                                     |


## Schedule (Example - 2 Days):


**Day 1:**

* **8:00 AM:** Arrive in Bangalore, check into your hotel.
* **9:00 AM:** Breakfast at a local South Indian restaurant (Idli, Dosa).
* **10:00 AM:** Visit Lalbagh Botanical Garden.
* **1:00 PM:** Lunch at a restaurant near Lalbagh. (Try South Indian Thali)
* **2:30 PM:** Visit Bangalore Palace.
* **5:00 PM:** Explore Commercial Street for shopping.
* **7:00 PM:** Dinner at a restaurant on MG Road (North Indian or Continental cuisine).
* **8:30 PM:** Relax at the hotel or explore the local area.

**Day 2:**

* **8:00 AM:** Breakfast at the hotel or a nearby cafe.
* **9:00 AM:** Visit Cubbon Park.
* **11:00 AM:** Visit Tipu Sultan's Summer Palace.
* **1:00 PM:** Lunch at a restaurant near Cubbon Park (Try street food like Pani Puri).
* **2:30 PM:** Visit ISKCON Temple.
* **4:30 PM:** Visit Vidhana Soudha.
* **6:00 PM:** Enjoy street food or a casual dinner near your hotel.
* **7:30 PM:** Depart from Bangalore.


**Food Preferences:** The schedule includes suggestions for South Indian and North Indian cuisine, as well as street food options. Adapt this to your preferences.  Bangalore offers diverse culinary options.


**Note:** This is a sample schedule.  Adjust it based on your interests and the duration of your trip.  Consider using public transportation or ride-sharing apps within Bangalore to save money and time.  Always confirm timings and opening hours for the places you plan to visit.